## Baseline_model and first simple model

In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
# Loading the dataset for our baseline model
df_baseline = pd.read_csv('/home/sergei/Desktop/LighthouseLabs/Re_enter_bootcamp/Final_Project/Austin_House_data/baseline_dataset.csv', index_col=None)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df_baseline.head(5)

,Unnamed: 0,new_price,latitude,longitude,propertyTaxRate,garageSpaces,hasAssociation,hasCooling,hasGarage,hasHeating,hasSpa,hasView,homeType,parkingSpaces,yearBuilt,numPriceChanges,numOfAccessibilityFeatures,numOfAppliances,numOfParkingFeatures,numOfPatioAndPorchFeatures,numOfSecurityFeatures,numOfWaterfrontFeatures,numOfWindowFeatures,numOfCommunityFeatures,lotSizeSqFt,livingAreaSqFt,numOfPrimarySchools,numOfElementarySchools,numOfMiddleSchools,numOfHighSchools,avgSchoolDistance,avgSchoolRating,avgSchoolSize,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories
0,0,757493.0,30.486408,-97.794724,2.21,0,True,True,False,True,False,False,Single Family,0,2014,13,0,4,2,0,0,0,0,0,10672.0,4564.0,1,0,1,1,3.266667,7.666667,1259,14,6.0,5,2
1,1,417108.0,30.494375,-97.796516,2.21,2,True,True,True,True,False,True,Single Family,2,2007,4,0,6,3,0,0,0,0,0,9060.0,3233.0,1,0,1,1,1.933333,8.333333,1481,16,4.0,5,2
2,2,201441.0,30.255707,-97.576958,1.98,2,False,True,True,True,False,False,Single Family,2,2006,3,1,2,2,1,0,0,0,0,7448.0,1511.0,1,0,1,1,2.433333,2.666667,1478,13,2.0,3,1
3,3,458883.0,30.495638,-97.797874,2.21,0,True,True,False,True,False,False,Single Family,0,2008,2,0,3,2,0,0,0,0,0,7666.0,2228.0,1,0,1,1,1.900000,8.333333,1481,16,2.0,3,1
4,4,664216.0,30.488775,-97.794899,2.21,2,True,True,True,True,False,False,Single Family,2,2013,2,0,4,3,0,0,0,0,0,8494.0,3494.0,1,0,1,1,3.300000,7.666667,1259,14,5.0,4,2


In [5]:
df_baseline.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
df_baseline.head(1)

,new_price,latitude,longitude,propertyTaxRate,garageSpaces,hasAssociation,hasCooling,hasGarage,hasHeating,hasSpa,hasView,homeType,parkingSpaces,yearBuilt,numPriceChanges,numOfAccessibilityFeatures,numOfAppliances,numOfParkingFeatures,numOfPatioAndPorchFeatures,numOfSecurityFeatures,numOfWaterfrontFeatures,numOfWindowFeatures,numOfCommunityFeatures,lotSizeSqFt,livingAreaSqFt,numOfPrimarySchools,numOfElementarySchools,numOfMiddleSchools,numOfHighSchools,avgSchoolDistance,avgSchoolRating,avgSchoolSize,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories
0,757493.0,30.486408,-97.794724,2.21,0,True,True,False,True,False,False,Single Family,0,2014,13,0,4,2,0,0,0,0,0,10672.0,4564.0,1,0,1,1,3.266667,7.666667,1259,14,6.0,5,2


In [7]:
X = df_baseline.drop('new_price', axis=1)

In [8]:
y = df_baseline.new_price
y_log=np.log(y)

In [9]:
# let's create a minimum baseline prediction that always predicts mean value of y and calculate MAE as a baseline measure
y_baseline = y.copy()

for i in range(len(y)):
    y_baseline[i] = y.mean()

In [10]:
# Calculate MAE for baseline prediction and we will see the future improvement for other models
print(mean_absolute_error(y, y_baseline))

182036.90327521067


In [11]:
# determine categorical and numerical features
numerical_ix = X.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X.select_dtypes(include=['object', 'bool']).columns

In [12]:
# define the data preparation for the columns
t = [('cat', OneHotEncoder(), categorical_ix), ('num', MinMaxScaler(), numerical_ix)]
col_transform = ColumnTransformer(transformers=t)

In [13]:
# Select random state
randomstate=30

In [14]:
# split the data
X_train,X_test,y_train,y_test=train_test_split(X,y_log,test_size=0.25,random_state=randomstate)

In [15]:
# define the model
model = LinearRegression()

In [16]:
# define the data preparation and modeling pipeline
pipeline = Pipeline(steps=[('prep',col_transform), ('m',model)])

In [17]:
# define the model cross-validation configuration
cv = KFold(n_splits=10, shuffle=True, random_state=randomstate)

In [18]:
# evaluate the pipeline using cross validation and calculate MAE as best metric for price prediction
score_R2 = cross_val_score(pipeline, X, y, scoring='r2', cv=cv, n_jobs=-1)
score_NMAE = cross_val_score(pipeline, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
score_RMSE = cross_val_score(pipeline, X, y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)

In [19]:
print(score_R2.mean())
print(abs(score_NMAE.mean()))
print(abs(score_RMSE.mean()))

0.520778989165867
124097.07935256604
181721.20090365023


In [20]:
# Predicting on log transformed target variable (less skewed target variable distribution) and finding actual MAE in $ when target variable inverse log transformed

pipeline.fit(X_train, y_train)
y_predict = pipeline.predict(X_test)

#R2, MAE and RMSE
print(r2_score(y_test, y_predict))
print(mean_absolute_error(np.exp(y_test), np.exp(y_predict)))
print(np.sqrt(mean_squared_error(np.exp(y_test), np.exp(y_predict))))

# Adjusted R2
n = X_test.shape[0]
p = X_test.shape[1]
R2 = r2_score(y_test, y_predict)
adj_R2 = 1 - ((1-R2)*(n-1)/(n-p-1))
print(adj_R2)

0.5680580727583858
113825.82641362768
184051.23353689475
0.5634989752151239


In [21]:
# Not bad for the first trained model. I assume this good result is achieved because of our thorough work with outliers in EDA.

## Modelling after the Feature engineering

In [22]:
# loading datasets
train = pd.read_csv('/home/sergei/Desktop/LighthouseLabs/Re_enter_bootcamp/Final_Project/Austin_House_data/train1_dataset.csv', index_col=0)
test = pd.read_csv('/home/sergei/Desktop/LighthouseLabs/Re_enter_bootcamp/Final_Project/Austin_House_data/test1_dataset.csv', index_col=0)

In [23]:
# Separating and log-transforming target variable
X_train = train.drop('new_price', axis=1)
X_test = test.drop('new_price', axis=1)
y_train = train.new_price
y_test = test.new_price
y_train_log=np.log(y_train)
y_test_log=np.log(y_test)

In [24]:
# determine categorical and numerical features
numerical_ix = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X_train.select_dtypes(include=['object']).columns

In [25]:
# define the data preparation for the columns
t = [('cat', OneHotEncoder(), categorical_ix), ('num', MinMaxScaler(), numerical_ix)]
col_transform = ColumnTransformer(transformers=t)

In [26]:
# define the model
model = LinearRegression()

In [27]:
# define the data preparation and modeling pipeline
pipeline = Pipeline(steps=[('prep',col_transform), ('m',model)])

In [28]:
# Predicting on log transformed target variable (less skewed target variable distribution) and finding actual MAE in $ when target variable inverse log transformed
pipeline.fit(X_train, y_train_log)
y_predict_log = pipeline.predict(X_test)

#R2, MAE and RMSE
print(r2_score(y_test_log, y_predict_log))
print(mean_absolute_error(np.exp(y_test_log), np.exp(y_predict_log)))
print(np.sqrt(mean_squared_error(np.exp(y_test_log), np.exp(y_predict_log))))

0.683609291113662
94760.78913532176
144752.5954272284


In [29]:
# adjusted R2
n = X_test.shape[0]
p = X_test.shape[1]
R2 = r2_score(y_test_log, y_predict_log)
adj_R2 = 1 - ((1-R2)*(n-1)/(n-p-1))
print(adj_R2)

0.6822819102552835


In [30]:
# The results improved - we already explain 2/3 of variation of the target vaiable and our Absolute Mean Error decreased by 17000$

## Modelling after NLP

In [31]:
# loading datasets
train = pd.read_csv('/home/sergei/Desktop/LighthouseLabs/Re_enter_bootcamp/Final_Project/Austin_House_data/train_nlp_dataset.csv', index_col=0)
test = pd.read_csv('/home/sergei/Desktop/LighthouseLabs/Re_enter_bootcamp/Final_Project/Austin_House_data/test_nlp_dataset.csv', index_col=0)

In [32]:
# Separating and log-transforming target variable
X_train = train.drop('new_price', axis=1)
X_test = test.drop('new_price', axis=1)
y_train = train.new_price
y_test = test.new_price
y_train_log=np.log(y_train)
y_test_log=np.log(y_test)

In [33]:
# determine categorical and numerical features
numerical_ix = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X_train.select_dtypes(include=['object']).columns

In [34]:
# define the data preparation for the columns
t = [('cat', OneHotEncoder(), categorical_ix), ('num', MinMaxScaler(), numerical_ix)]
col_transform = ColumnTransformer(transformers=t)

In [35]:
# define the model
model = LinearRegression()

In [36]:
# define the data preparation and modeling pipeline
pipeline = Pipeline(steps=[('prep',col_transform), ('m',model)])

In [37]:
pipeline.fit(X_train, y_train_log)
y_predict_log = pipeline.predict(X_test)

#R2, MAE and RMSE
print(r2_score(y_test_log, y_predict_log))
print(mean_absolute_error(np.exp(y_test_log), np.exp(y_predict_log)))
print(np.sqrt(mean_squared_error(np.exp(y_test_log), np.exp(y_predict_log))))

0.6835462678523603
94612.35940314812
145084.04197734737


In [38]:
# adjusted R2
n = X_test.shape[0]
p = X_test.shape[1]
R2 = r2_score(y_test_log, y_predict_log)
adj_R2 = 1 - ((1-R2)*(n-1)/(n-p-1))
print(adj_R2)

0.6691305908184897


In [39]:
# The results are disappointing at first since our model has not improved, but from my point of view it is because of the limitations of linear regression model and we have more than 100 features.
# Let's try another more complex one, for example SVM with hyperparameter tuning

# Modelling SVM with hyperparameter tuning using GridSearchCV

In [40]:
'''# Parameter Tuning

param_grid = {'m__kernel' : ['rbf'],
              'm__gamma' : ['auto'],
              'm__C':[75, 100, 125],
              'm__epsilon':[0.05, 0.1, 0.15],
              'm__tol':[0.001,0.005,0.01,0.05]
              }

model = SVR()

pipeline = Pipeline(steps=[('prep',col_transform), ('m',model)])

grid_search = GridSearchCV(pipeline, param_grid, verbose=10, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train_log)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

'# Parameter Tuning\n\nparam_grid = {\'m__kernel\' : [\'rbf\'],\n              \'m__gamma\' : [\'auto\'],\n              \'m__C\':[75, 100, 125],\n              \'m__epsilon\':[0.05, 0.1, 0.15],\n              \'m__tol\':[0.001,0.005,0.01,0.05]\n              }\n\nmodel = SVR()\n\npipeline = Pipeline(steps=[(\'prep\',col_transform), (\'m\',model)])\n\ngrid_search = GridSearchCV(pipeline, param_grid, verbose=10, scoring=\'neg_mean_absolute_error\', cv=5, n_jobs=-1)\n\ngrid_search.fit(X_train, y_train_log)\n\ngrid_search.best_estimator_\n\nprint("Best parameters set found on train set: \n")\nprint(grid_search.best_params_)\nprint("\nGrid scores on train set:\n")\nmeans = grid_search.cv_results_[\'mean_test_score\']\nstds = grid_search.cv_results_[\'std_test_score\']\nfor mean, std, params in zip(means, stds, grid_search.cv_results_[\'params\']):\n    print("%0.3f (+/-%0.03f) for %r"\n              % (mean, std * 2, params))'

In [41]:
# grid_search.get_params().keys()

In [42]:
params = {'tol': 0.005, 'kernel' : 'rbf', 'C' : 125, 'epsilon' : 0.1, 'gamma':'auto'}
model = SVR(**params, verbose=True)
pipeline = Pipeline(steps=[('prep',col_transform), ('m',model)])
pipeline.fit(X_train, y_train_log)
y_predict_log = pipeline.predict(X_test)

#R2 and adjusted R2, MAE, RMSE
print(r2_score(y_test_log, y_predict_log))
print(mean_absolute_error(np.exp(y_test_log), np.exp(y_predict_log)))
print(np.sqrt(mean_squared_error(np.exp(y_test_log), np.exp(y_predict_log))))

[LibSVM]0.7031241551250526
90108.05893086368
138910.83454798165


In [43]:
# adjusted R2
n = X_test.shape[0]
p = X_test.shape[1]
R2 = r2_score(y_test_log, y_predict_log)
adj_R2 = 1 - ((1-R2)*(n-1)/(n-p-1))
print(adj_R2)

0.6896003256861314


In [44]:
# Slight improvement, but still not the improvement we were hoping for. Let's try totally different complex model - XGBoost

# Modelling XGBoost with hyperparameter tuning using GridSearchCV

In [45]:
'''# Parameter Tuning

param_grid = {'m__learning_rate' : [0.05, 0.1, 0.15],
              'm__max_depth': [7, 8, 9],
              'm__gamma':[10, 1, 0.1],
              'm__subsample':[0.9, 0.95, 1],
              'm__colsample_bytree':[0.8, 0.85, 0.9],
              'm__alpha':[1, 5, 10],
              'm__lambda':[0.5, 1, 1.5],
              'm__n_estimators':[90, 100, 110]}
            
model = xgb.XGBRegressor(eval_metric='mae')

pipeline = Pipeline(steps=[('prep',col_transform), ('m',model)])

grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv = 5, 
                                   verbose=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

'# Parameter Tuning\n\nparam_grid = {\'m__learning_rate\' : [0.05, 0.1, 0.15],\n              \'m__max_depth\': [7, 8, 9],\n              \'m__gamma\':[10, 1, 0.1],\n              \'m__subsample\':[0.9, 0.95, 1],\n              \'m__colsample_bytree\':[0.8, 0.85, 0.9],\n              \'m__alpha\':[1, 5, 10],\n              \'m__lambda\':[0.5, 1, 1.5],\n              \'m__n_estimators\':[90, 100, 110]}\n            \nmodel = xgb.XGBRegressor(eval_metric=\'mae\')\n\npipeline = Pipeline(steps=[(\'prep\',col_transform), (\'m\',model)])\n\ngrid_search = GridSearchCV(pipeline, param_grid=param_grid, cv = 5, \n                                   verbose=5, n_jobs=-1)\ngrid_search.fit(X_train, y_train)\n\ngrid_search.best_estimator_\n\nprint("Best parameters set found on train set: \n")\nprint(grid_search.best_params_)\nprint("\nGrid scores on train set:\n")\nmeans = grid_search.cv_results_[\'mean_test_score\']\nstds = grid_search.cv_results_[\'std_test_score\']\nfor mean, std, params in zip(me

In [46]:
params = {'n_estimators': 97, 
          'learning_rate': 0.09,
          'max_depth': 8,
          'gamma': 1, 
          'subsample': 0.95, 
          'colsample_bytree': 0.85,
          'alpha': 2,
          'lambda': 1}
model = xgb.XGBRegressor(**params, eval_metric='mae')
pipeline = Pipeline(steps=[('prep',col_transform), ('m',model)])

# We don't need to log transform our target variable since it does not improve the results for XGBoost
pipeline.fit(X_train, y_train)
y_predict = pipeline.predict(X_test)

#R2 and adjusted R2, MAE and RMSE
print(r2_score(y_test, y_predict))

print(mean_absolute_error(y_test, y_predict))
print(np.sqrt(mean_squared_error(y_test, y_predict)))

0.7761726840973217
76501.64876379773
122153.38319114472


In [47]:
# adjusted R2
n = X_test.shape[0]
p = X_test.shape[1]
R2 = r2_score(y_test_log, y_predict_log)
R2 = r2_score(y_test, y_predict)
adj_R2 = 1 - ((1-R2)*(n-1)/(n-p-1))
print(adj_R2)

0.7659764943557332


In [48]:
# Much better result with XGBoost

# Adding MAPIE package for prediction intervals and confidence levels

In [49]:
# estimate the prediction intervals instead of a prediction of a certain price per house, which is, frankly speaking, seems as an unrealistic task.

from mapie.regression import MapieRegressor
from mapie.metrics import regression_coverage_score

In [50]:
# since MAPIE does not work with pipelines yet, let's transform our features first
X_train = col_transform.fit_transform(X_train)
X_test = col_transform.fit_transform(X_test)

In [51]:
# we define our model with best parameters figured previously
params = {'n_estimators': 97, 
          'learning_rate': 0.09,
          'max_depth': 8,
          'gamma': 1, 
          'subsample': 0.95, 
          'colsample_bytree': 0.85,
          'alpha': 2,
          'lambda': 1}

model = xgb.XGBRegressor(**params, eval_metric='mae')

In [52]:
mapie = MapieRegressor(estimator=model, method="base", cv=5, n_jobs=-1)
alpha = 0.25

In [53]:
mapie.fit(X_train, y_train)

MapieRegressor(cv=5,
               estimator=XGBRegressor(alpha=2, base_score=None, booster=None,
                                      colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=0.85,
                                      enable_categorical=False,
                                      eval_metric='mae', gamma=1, gpu_id=None,
                                      importance_type=None,
                                      interaction_constraints=None, lambda=1,
                                      learning_rate=0.09, max_delta_step=None,
                                      max_depth=8, min_child_weight=None,
                                      missing=nan, monotone_constraints=None,
                                      n_estimators=97, n_jobs=None,
                                      num_parallel_tree=None, predictor=None,
                                      random_state=None, re

In [54]:
# Get interval predictions on test data, with alpha=05%
y_test_pred_interval = pd.DataFrame(mapie.predict(X_test, alpha=alpha)[1].reshape(-1,2), columns=["left", "right"]) 
# we reshaped the output of MAPIE for better visual representation

In [55]:
y_test_pred_interval.round(2) # this is interval for our house price prediction with 75% confidence that our predicted price will be within it

,left,right
0,354657.25,548336.12
1,697784.00,891462.88
2,973731.81,1167410.69
3,482099.25,675778.12
4,425140.19,618819.06
...,...,...
3347,241306.38,434985.25
3348,233692.94,427371.81
3349,706088.44,899767.31
3350,252992.34,446671.22


In [56]:
# Not very applicable in this case, but that is because of the current perfomance of our model that gives only slightly above 75% of variance of target variable explained